In [11]:
# Install required libraries
!pip install torch torchvision pytorch-lightning

# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, Subset
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report

In [12]:
import os
import tarfile
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Step 2: Unzip the Mini-ImageNet tar files
data_dir = "/content/mini-imagenet"
os.makedirs(data_dir, exist_ok=True)

# Unzip train, val, and test tar files
for split in ["train", "val", "test"]:
    tar_path = f"/content/{split}.tar"
    with tarfile.open(tar_path, "r") as tar:
        tar.extractall(data_dir)
    print(f"Extracted {split}.tar to {data_dir}/{split}")

# Define transforms (resize to 96x96 as required)
transform = transforms.Compose([
    transforms.Resize((96, 96)),  # Resize to 96x96 as per requirement
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

# Load the datasets using ImageFolder (assumes folder structure: data_dir/split/class_name/images)
train_dataset = ImageFolder(root=f"{data_dir}/train", transform=transform)
val_dataset = ImageFolder(root=f"{data_dir}/val", transform=transform)
test_dataset = ImageFolder(root=f"{data_dir}/test", transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# Verify dataset sizes (Mini-ImageNet: 100 classes, 500 train, 100 test per class)
print(f"Train dataset size: {len(train_dataset)}")  # Should be ~50,000 (100 * 500)
print(f"Val dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")  # Should be ~10,000 (100 * 100)

Extracted train.tar to /content/mini-imagenet/train
Extracted val.tar to /content/mini-imagenet/val
Extracted test.tar to /content/mini-imagenet/test
Train dataset size: 38400
Val dataset size: 12000
Test dataset size: 12000


In [13]:
# Step 3: Select Two Pre-trained Models
import torch
import torch.nn as nn
from torchvision import models

# Load pre-trained models
resnet18 = models.resnet18(pretrained=True)
mobilenet_v2 = models.mobilenet_v2(pretrained=True)

# Modify the final layer for Mini-ImageNet (100 classes)
num_classes = 100  # Mini-ImageNet has 100 classes

# For ResNet18
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)

# For MobileNetV2
mobilenet_v2.classifier[1] = nn.Linear(mobilenet_v2.classifier[1].in_features, num_classes)

# Move models to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = resnet18.to(device)
mobilenet_v2 = mobilenet_v2.to(device)

# Verify the modified models
print("Modified ResNet18 last layer:", resnet18.fc)
print("Modified MobileNetV2 last layer:", mobilenet_v2.classifier[1])

Modified ResNet18 last layer: Linear(in_features=512, out_features=100, bias=True)
Modified MobileNetV2 last layer: Linear(in_features=1280, out_features=100, bias=True)


In [14]:
# Step 4: Define the Fine-tuning Process
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score

# Define a PyTorch Lightning module for fine-tuning
class ImageClassifier(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy_score(y.cpu(), preds.cpu())
        self.log("val_loss", loss)
        self.log("val_acc", acc)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy_score(y.cpu(), preds.cpu())
        self.log("test_loss", loss)
        self.log("test_acc", acc)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return optimizer

# Initialize models for fine-tuning
resnet_trainer = ImageClassifier(resnet18, learning_rate=1e-3)
mobilenet_trainer = ImageClassifier(mobilenet_v2, learning_rate=1e-3)

# Verify the setup
print("ResNet18 Trainer initialized with learning rate:", resnet_trainer.learning_rate)
print("MobileNetV2 Trainer initialized with learning rate:", mobilenet_trainer.learning_rate)

ResNet18 Trainer initialized with learning rate: 0.001
MobileNetV2 Trainer initialized with learning rate: 0.001


In [15]:
# Step 5: Fine-tune the Models on the Mini-ImageNet Training Set
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

# Define checkpoint callback to save the best model based on validation accuracy
checkpoint_callback = ModelCheckpoint(monitor="val_acc", mode="max", save_top_k=1)

# Fine-tune ResNet18
trainer_resnet = pl.Trainer(
    max_epochs=5,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,  # Explicitly set to 1 for both GPU and CPU to avoid parsing issues
    callbacks=[checkpoint_callback]
)
trainer_resnet.fit(resnet_trainer, train_loader, val_loader)

# Fine-tune MobileNetV2
trainer_mobilenet = pl.Trainer(
    max_epochs=5,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,  # Explicitly set to 1 for both GPU and CPU to avoid parsing issues
    callbacks=[checkpoint_callback]
)
trainer_mobilenet.fit(mobilenet_trainer, train_loader, val_loader)

# Print the best model paths
print("Best ResNet18 model saved at:", checkpoint_callback.best_model_path)
print("Best MobileNetV2 model saved at:", checkpoint_callback.best_model_path)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | ResNet           | 11.2 M | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.911    Total estimated model params size (MB)
69        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


Best ResNet18 model saved at: /content/lightning_logs/version_0/checkpoints/epoch=1-step=2400.ckpt
Best MobileNetV2 model saved at: /content/lightning_logs/version_0/checkpoints/epoch=1-step=2400.ckpt


In [17]:
# Step 6: Establish Baseline Performance on the Uncompressed Test Set
import torch
from sklearn.metrics import accuracy_score, classification_report

# Check if the best model path exists and load the state dictionaries
if checkpoint_callback.best_model_path:
    resnet18.load_state_dict(torch.load(checkpoint_callback.best_model_path, map_location=device), strict=False)
    mobilenet_v2.load_state_dict(torch.load(checkpoint_callback.best_model_path, map_location=device), strict=False)
else:
    print("No checkpoint found. Using initial model weights.")
    # Optionally, you can skip evaluation or use the untrained models

# Move models to evaluation mode and device
resnet18.eval()
mobilenet_v2.eval()
resnet18 = resnet18.to(device)
mobilenet_v2 = mobilenet_v2.to(device)

# Evaluate models on the test set
def evaluate_model(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for x, y in data_loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())
    return all_preds, all_labels

# Get predictions and labels for both models
resnet_preds, resnet_labels = evaluate_model(resnet18, test_loader)
mobilenet_preds, mobilenet_labels = evaluate_model(mobilenet_v2, test_loader)

# Calculate accuracy
resnet_acc = accuracy_score(resnet_labels, resnet_preds)
mobilenet_acc = accuracy_score(mobilenet_labels, mobilenet_preds)

# Print accuracy
print(f"ResNet18 Test Accuracy: {resnet_acc:.4f}")
print(f"MobileNetV2 Test Accuracy: {mobilenet_acc:.4f}")

# Print detailed classification report
print("\nResNet18 Classification Report:")
print(classification_report(resnet_labels, resnet_preds))

print("\nMobileNetV2 Classification Report:")
print(classification_report(mobilenet_labels, mobilenet_preds))

ResNet18 Test Accuracy: 0.0048
MobileNetV2 Test Accuracy: 0.0047

ResNet18 Classification Report:
              precision    recall  f1-score   support

           0       0.09      0.02      0.03       600
           1       0.08      0.01      0.02       600
           2       0.03      0.00      0.01       600
           3       0.01      0.00      0.00       600
           4       0.02      0.00      0.01       600
           5       0.10      0.00      0.00       600
           6       0.03      0.01      0.01       600
           7       0.01      0.00      0.00       600
           8       0.04      0.01      0.01       600
           9       0.00      0.00      0.00       600
          10       0.00      0.00      0.00       600
          11       0.00      0.00      0.00       600
          12       0.01      0.01      0.01       600
          13       0.00      0.00      0.00       600
          14       0.01      0.00      0.01       600
          15       0.02      0.01    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.p

In [18]:
# Step 7: Document Model Architecture, Hyperparameters, and Training Process
import torch
from torchvision import models

# Document model architecture
print("ResNet18 Architecture:")
print(resnet18)
print("\nMobileNetV2 Architecture:")
print(mobilenet_v2)

# Document hyperparameters and training process
documentation = """
### Model Selection and Fine-tuning Documentation

#### Models Selected:
- **ResNet18**: A lightweight residual network with 18 layers, pre-trained on ImageNet.
- **MobileNetV2**: A mobile-optimized network with inverted residuals, pre-trained on ImageNet.

#### Hyperparameters:
- **Learning Rate**: 1e-3 (Adam optimizer)
- **Batch Size**: 32
- **Epochs**: 5
- **Loss Function**: CrossEntropyLoss
- **Optimizer**: Adam

#### Training Process:
- **Dataset**: Mini-ImageNet (100 classes, 500 training images per class, 100 validation/test images per class)
- **Preprocessing**: Resized images to 96x96 pixels, normalized with ImageNet mean [0.485, 0.456, 0.406] and std [0.229, 0.224, 0.225].
- **Validation Split**: Used provided validation set from Mini-ImageNet val.tar.
- **Fine-tuning Strategy**: Modified the final fully connected layer for 100 classes. Trained all layers with Adam optimizer.
- **Validation Strategy**: Monitored validation accuracy with ModelCheckpoint to save the best model.
- **Hardware**: Utilized GPU if available, otherwise CPU.

#### Baseline Performance:
- **ResNet18 Test Accuracy**: {resnet_acc:.4f}
- **MobileNetV2 Test Accuracy**: {mobilenet_acc:.4f}
- **Detailed Metrics**: Classification reports provided below.
""".format(resnet_acc=resnet_acc, mobilenet_acc=mobilenet_acc)

# Print documentation
print(documentation)

# Save documentation to a file
with open("fine_tuning_documentation.md", "w") as f:
    f.write(documentation)

# Append classification reports to the documentation file
with open("fine_tuning_documentation.md", "a") as f:
    f.write("\n### ResNet18 Classification Report:\n")
    f.write(classification_report(resnet_labels, resnet_preds))
    f.write("\n### MobileNetV2 Classification Report:\n")
    f.write(classification_report(mobilenet_labels, mobilenet_preds))

ResNet18 Architecture:
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu)

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.p

In [20]:
# Step 8: Implement Proper Validation Strategies
# Validation strategy already implemented in Steps 2 and 5:
# - Used the Mini-ImageNet validation set (val.tar) as a separate validation dataset.
# - Monitored validation accuracy during training with PyTorch Lightning.
# - Saved the best model using ModelCheckpoint based on val_acc.

# Document the validation strategy
validation_documentation = """
### Validation Strategy Documentation

#### Validation Dataset:
- Source: Mini-ImageNet validation set (val.tar), containing 100 images per class for 100 classes (10,000 images total).
- Preprocessing: Same as training set—resized to 96x96 pixels, normalized with ImageNet mean [0.485, 0.456, 0.406] and std [0.229, 0.224, 0.225].

#### Validation Process:
- Frequency: Validation was performed after each epoch during training.
- Metric Monitored: Validation accuracy (`val_acc`) was computed using sklearn.metrics.accuracy_score.
- Checkpointing: Used PyTorch Lightning’s ModelCheckpoint callback to save the model with the highest validation accuracy (`monitor="val_acc", mode="max", save_top_k=1`).

#### Purpose:
- Ensured the model generalizes well to unseen data and prevents overfitting by monitoring validation accuracy.
- The best model (based on validation accuracy) was saved and used for testing in Step 6.

#### Validation Results:
- Best validation accuracy for ResNet18 and MobileNetV2 was logged during training (visible in PyTorch Lightning logs).
- Final test performance on the uncompressed test set reflects the effectiveness of this strategy (see Step 6).
"""

# Print the validation strategy documentation
print(validation_documentation)

# Append to the existing documentation file
with open("fine_tuning_documentation.md", "a") as f:
    f.write("\n")
    f.write(validation_documentation)


### Validation Strategy Documentation

#### Validation Dataset:
- Source: Mini-ImageNet validation set (val.tar), containing 100 images per class for 100 classes (10,000 images total).
- Preprocessing: Same as training set—resized to 96x96 pixels, normalized with ImageNet mean [0.485, 0.456, 0.406] and std [0.229, 0.224, 0.225].

#### Validation Process:
- Frequency: Validation was performed after each epoch during training.
- Metric Monitored: Validation accuracy (`val_acc`) was computed using sklearn.metrics.accuracy_score.
- Checkpointing: Used PyTorch Lightning’s ModelCheckpoint callback to save the model with the highest validation accuracy (`monitor="val_acc", mode="max", save_top_k=1`).

#### Purpose:
- Ensured the model generalizes well to unseen data and prevents overfitting by monitoring validation accuracy.
- The best model (based on validation accuracy) was saved and used for testing in Step 6.

#### Validation Results:
- Best validation accuracy for ResNet18 and MobileNet

In [21]:
# Step 9: Save and Share Your Work
import torch

# Save the fine-tuned models
torch.save(resnet18.state_dict(), "resnet18_finetuned.pth")
torch.save(mobilenet_v2.state_dict(), "mobilenet_v2_finetuned.pth")

# Verify file paths
print("ResNet18 model saved at: resnet18_finetuned.pth")
print("MobileNetV2 model saved at: mobilenet_v2_finetuned.pth")

# Save the documentation file (already created in Step 7 and appended in Step 8)
# No additional save needed, but confirm it exists
print("Documentation saved at: fine_tuning_documentation.md")

# Instructions for sharing with the team
sharing_instructions = """
### Instructions to Share Your Work

1. **Download Files from Colab**:
   - Right-click on `resnet18_finetuned.pth`, `mobilenet_v2_finetuned.pth`, and `fine_tuning_documentation.md` in the Colab file explorer.
   - Select 'Download' to save them to your local machine.

2. **Create or Update GitHub Repository**:
   - Navigate to your team's GitHub repository (e.g., the one specified for EE-413 Project 2).
   - Create a new folder named `model_outputs` if it doesn’t exist.
   - Upload the downloaded files (`resnet18_finetuned.pth`, `mobilenet_v2_finetuned.pth`, `fine_tuning_documentation.md`) to the `model_outputs` folder.

3. **Commit and Push Changes**:
   - Use Git commands or the GitHub web interface to commit the files with a message (e.g., "Added fine-tuned models and documentation for Mini-ImageNet").
   - Push the changes to the remote repository.

4. **Share with Team**:
   - Notify your team via the project communication channel (e.g., email, Slack) with the repository link and a brief description of the files.
   - Ensure all team members have access to the repository.

5. **Colab Notebook**:
   - Download your Colab notebook (File > Download .ipynb).
   - Upload the `.ipynb` file to the repository root or a `notebooks` folder for reference.
"""

# Print sharing instructions
print(sharing_instructions)

# Optional: Download files directly from Colab (if needed)
from google.colab import files
files.download("resnet18_finetuned.pth")
files.download("mobilenet_v2_finetuned.pth")
files.download("fine_tuning_documentation.md")

ResNet18 model saved at: resnet18_finetuned.pth
MobileNetV2 model saved at: mobilenet_v2_finetuned.pth
Documentation saved at: fine_tuning_documentation.md

### Instructions to Share Your Work

1. **Download Files from Colab**:
   - Right-click on `resnet18_finetuned.pth`, `mobilenet_v2_finetuned.pth`, and `fine_tuning_documentation.md` in the Colab file explorer.
   - Select 'Download' to save them to your local machine.

2. **Create or Update GitHub Repository**:
   - Navigate to your team's GitHub repository (e.g., the one specified for EE-413 Project 2).
   - Create a new folder named `model_outputs` if it doesn’t exist.
   - Upload the downloaded files (`resnet18_finetuned.pth`, `mobilenet_v2_finetuned.pth`, `fine_tuning_documentation.md`) to the `model_outputs` folder.

3. **Commit and Push Changes**:
   - Use Git commands or the GitHub web interface to commit the files with a message (e.g., "Added fine-tuned models and documentation for Mini-ImageNet").
   - Push the changes to 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
# Step 9: Save and Share Your Work (Updated)
import torch
from google.colab import drive
import os

# Save the fine-tuned models
torch.save(resnet18.state_dict(), "resnet18_finetuned.pth")
torch.save(mobilenet_v2.state_dict(), "mobilenet_v2_finetuned.pth")

# Verify file paths
print("ResNet18 model saved at: resnet18_finetuned.pth")
print("MobileNetV2 model saved at: mobilenet_v2_finetuned.pth")
print("Documentation saved at: fine_tuning_documentation.md")

# Mount Google Drive to save large files
drive.mount('/content/drive')

# Define a folder in Google Drive to save the files
save_dir = '/content/drive/My Drive/EE413_Project_Files'
os.makedirs(save_dir, exist_ok=True)

# Copy files to Google Drive
import shutil

# Copy ResNet18 model
shutil.copy("resnet18_finetuned.pth", save_dir)
print(f"ResNet18 model copied to: {save_dir}/resnet18_finetuned.pth")

# Copy MobileNetV2 model
shutil.copy("mobilenet_v2_finetuned.pth", save_dir)
print(f"MobileNetV2 model copied to: {save_dir}/mobilenet_v2_finetuned.pth")

# Copy documentation
shutil.copy("fine_tuning_documentation.md", save_dir)
print(f"Documentation copied to: {save_dir}/fine_tuning_documentation.md")




ResNet18 model saved at: resnet18_finetuned.pth
MobileNetV2 model saved at: mobilenet_v2_finetuned.pth
Documentation saved at: fine_tuning_documentation.md
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ResNet18 model copied to: /content/drive/My Drive/EE413_Project_Files/resnet18_finetuned.pth
MobileNetV2 model copied to: /content/drive/My Drive/EE413_Project_Files/mobilenet_v2_finetuned.pth
Documentation copied to: /content/drive/My Drive/EE413_Project_Files/fine_tuning_documentation.md
